In [1]:
import cv2
import glob
import os
import numpy as np
import json

In [2]:
def get_single_frame(vid_path, frame_idx=10):
    '''
    Extract a single frame from a video
    Args:
        vid_path: video path
        frame_idx: the frame index 
    '''
    cap = cv2.VideoCapture(vid_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    cap.release()

    return frame

In [3]:
def load_rois(json_path="camera_rois.json"):
    if not os.path.exists(json_path):
        print("Error: ROI file not found.")
        return {}
        
    with open(json_path, 'r') as f:
        roi_data = json.load(f)
    
    return roi_data

In [4]:
def quantify_camera_shift(test_img, template, x_base, y_base):
    ''' 
    Template Matching:
    We search for the 'template' inside 'test_img'.
    TM_CCOEFF_NORMED is best for lighting invariance.

    Args:
    test_img: test image
    template: ROI of base image
    x_base: x location of template 
    y_base: y location of template 
    '''
    # Template Matching
    # We search for the 'template' inside 'test_img'
    # TM_CCOEFF_NORMED is best for lighting invariance.
    res = cv2.matchTemplate(test_img, template, cv2.TM_CCOEFF_NORMED)
    
    # Get the location of the best match
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    
    # For TM_CCOEFF_NORMED, the best match is the maximum value (max_loc)
    # max_loc is (x, y) top-left corner
    x_curr, y_curr = max_loc

    # 6. Calculate Shift
    shift_x = x_curr - x_base
    shift_y = y_curr - y_base
    
    # Confidence score (0 to 1). If < 0.8, the match might be wrong (e.g. object blocked).
    confidence = max_val 

    status_x_str = f"{shift_x:+d} px".ljust(8)
    status_y_str = f"{shift_y:+d} px".ljust(8)

    return status_x_str, status_y_str, confidence

In [5]:
# Setting
cameras = {'camTo', 'camTL', 'camTR', 'camBL', 'camBR'}
frame_idx = 10
rois_path = '/media/yiting/NewVolume/Data/Videos/Camera_Alignment/ROIs/2025-12-18_camera_rois.json'
video_folder_dir_base = '/media/yiting/NewVolume/Data/Videos/Camera_Alignment/2025-12-18/cameras/2025-12-18_11-22-05_062292'
video_folder_dir_curr = '/media/yiting/NewVolume/Data/Videos/Camera_Alignment/2025-12-18/cameras/2025-12-18_11-22-05_062292'

# Load ROIs
camera_rois = load_rois(rois_path)

for camera in cameras:
    # Load baseline frame
    video_path_base = os.path.join(video_folder_dir_base, f'{camera}-orig.mp4')
    frame_base = get_single_frame(video_path_base, frame_idx=frame_idx)
    gray_base = cv2.cvtColor(frame_base, cv2.COLOR_BGR2GRAY)

    # Create the template image
    x_base, y_base, w, h = camera_rois[camera]
    template = gray_base[y_base:y_base+h, x_base:x_base+w]

    # Load current frame
    video_path_curr = os.path.join(video_folder_dir_curr, f'{camera}-orig.mp4')
    frame_curr = get_single_frame(video_path_curr, frame_idx=frame_idx)
    gray_curr = cv2.cvtColor(frame_curr, cv2.COLOR_BGR2GRAY)

    # Template matching
    status_x_str, status_y_str, confidence = quantify_camera_shift(gray_curr, template, x_base, y_base)
    print(f"{camera} | {status_x_str} | {status_y_str} | {confidence:.2f}")

camTo | +0 px    | +0 px    | 1.00
camBL | +0 px    | +0 px    | 1.00
camTR | +0 px    | +0 px    | 1.00
camTL | +0 px    | +0 px    | 1.00
camBR | +0 px    | +0 px    | 1.00
